## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

In [ ]:
from sagemaker import get_execution_role
role = get_execution_role()

In [32]:
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='SagemakerFullAccess')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name Tony to get Role path.


In [91]:
#TODO: Declare your model training hyperparameter.
hyperparameters = {
    'batch_size': 128, 
    'lr': 0.001, 
    'epochs': 10, 
}


In [92]:
import uuid
bucket = "awscapstone"#sagemaker.Session().default_bucket()
prefix = 'resnet'

checkpoint_suffix = str(uuid.uuid4())[:8]
checkpoint_s3_path = 's3://{}/checkpoint-{}-{}'.format(bucket,prefix,checkpoint_suffix)
checkpoint_local_path='/opt/ml/checkpoints'

print('Checkpointing Path: {}'.format(checkpoint_s3_path))

use_spot_instances = True
max_run = 144000    #4 hours per epoch, 10 epochs
max_wait = 150000 if use_spot_instances else None

Checkpointing Path: s3://awscapstone/checkpoint-resnet-dc9edab3


In [93]:
#TODO: Create your training estimator
estimator = PyTorch(
    entry_point='train.py',
    base_job_name='capstone',
    role=role,
    instance_count=4,
    instance_type='ml.m5.2xlarge',
    framework_version='1.12.0',
    py_version='py38',
    checkpoint_s3_uri=checkpoint_s3_path,
    checkpoint_local_path=checkpoint_local_path,
    hyperparameters=hyperparameters,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait
)

In [94]:
# TODO: Fit your estimator
os.environ['SM_CHANNEL_TRAIN']='s3://awscapstone/train_data/'
os.environ['SM_CHANNEL_TEST']='s3://awscapstone/test_data/'
os.environ['SM_MODEL_DIR']='s3://awscapstone/'
estimator.fit({"train": os.environ['SM_CHANNEL_TRAIN'], "test": os.environ['SM_CHANNEL_TEST'] }, wait=True)

2022-12-04 06:03:31 Starting - Starting the training job...
2022-12-04 06:03:56 Starting - Preparing the instances for trainingProfilerReport-1670133811: InProgress
......
2022-12-04 06:04:56 Downloading - Downloading input data............
2022-12-04 06:06:57 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-04 06:06:59,877 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2022-12-04 06:06:59,879 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-12-04 06:06:59,888 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2022-12-04 06:06:59,896 sagemaker_pytorch_container.training INFO     Invoking user training script.
bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2022-12-04 0